In [ ]:
import pandas as pd
import numpy as np
# import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt

In [3]:
# data source: https://www.nhtsa.gov/research-data/fatality-analysis-reporting-system-fars
# read data from Data files and put them into a list of dataframes
def readAccidentData():
    FileList = [ "FARS1975","FARS1976","FARS1977","FARS1978","FARS1979","FARS1980","FARS1981","FARS1982"
            ,"FARS1983","FARS1984","FARS1985","FARS1986","FARS1987","FARS1988","FARS1989","FARS1990"
            ,"FARS1991","FARS1992","FARS1993","FARS1994","FARS1995","FARS1996","FARS1997","FARS1998"
            ,"FARS1999","FARS2000","FARS2001","FARS2002","FARS2003","FARS2004","FARS2005","FARS2006"
            ,"FARS2007","FARS2008","FARS2009","FARS2010","FARS2011","FARS2012","FARS2013","FARS2014"
            ,"FARS2015"
           ]
    dfList = []
    print ("Reading accident data for each year from 1975 to 2015.")
    for i in FileList:
        tmpfilepath = "../Data/" + i + "/accident.csv"
        # print ("Reading ", tmpfilepath)
        tmpdf = pd.read_csv(tmpfilepath)
        # tmpSkimmedDf = tmpdf[['STATE', 'COUNTY',"FATALS"]].copy()
        dfList.append( pd.read_csv(tmpfilepath) )
        # dfList.append(tmpSkimmedDf)
    return dfList

# get the county codes, explanation: https://en.wikipedia.org/wiki/FIPS_county_code
def addcountyCode(dfList):
    for i in range(len(dfList)):
        dfList[i]['STATE'] = dfList[i]['STATE'].apply(lambda x: int(x))
        dfList[i]['STATE'] = dfList[i]['STATE'].apply(lambda x: '{0:0>2}'.format(x))
        dfList[i]['COUNTY'] = dfList[i]['COUNTY'].apply(lambda x: int(x))
        dfList[i]["COUNTY"] = dfList[i]["COUNTY"].apply(lambda x: '{0:0>3}'.format(x))
        dfList[i]["COUNTYCODE"] = dfList[i]["STATE"].astype(str)+dfList[i]["COUNTY"].astype(str)
        #print (dfList[i]['COUNTYCODE'])
    return dfList

# remove all the unnecessary columns, keep only countycode and deaths per year
def cleanDataFrame(myDfList_AccidentDataAll):
    dfList = []
    for df in myDfList_AccidentDataAll:
        tmp = df[["COUNTYCODE","FATALS"]].copy()
        tmpGrouped = tmp.groupby(['COUNTYCODE'], as_index=False).sum()
        dfList.append(tmpGrouped)
    return dfList
    

myDfList_AccidentDataAll = readAccidentData()
myDfList_AccidentDataAll = addcountyCode(myDfList_AccidentDataAll)
myDfList_AccidentData = cleanDataFrame(myDfList_AccidentDataAll)
del myDfList_AccidentDataAll

Reading accident data for each year from 1975 to 2015.


In [4]:
# read population data from 1969 to 2015 at county level
# data source: https://seer.cancer.gov/popdata/download.html#19
def readPopData():
    filename = "../Data/us.1969_2015.19ages.txt"
    print ("Reading ",filename, ": yearly county level population data from 1969 to 2015")
    f = open(filename,"r")
    L = f.read()
    L = L.split()
    
    # extract only year, county, and population information
    year = []
    county = []
    population = []
    for x in L:
        year.append(int(x[:4]))
        county.append( str(x[6:11]) )
        population.append(int(x[18:]))
    myDf = pd.DataFrame({"YEAR" : year, "COUNTYCODE":county,"POPULATION":population})
    return myDf

def cleanPopData(df_raw):
    tmp = df_raw.groupby(['COUNTYCODE','YEAR'], as_index=False).sum()
    return tmp

def createPopList(myDf_PopulationData):
    yearList = list(range(1975,2016))
    dfList = []
    for y in yearList:
        tmp = myDf_PopulationData[myDf_PopulationData["YEAR"] == y]
        dfList.append( tmp )
    return dfList  
    
    
myDf_RawPopulationData = readPopData()
myDf_PopulationData = cleanPopData(myDf_RawPopulationData)
myDfList_PopData = createPopList(myDf_PopulationData)

Reading  ../Data/us.1969_2015.19ages.txt : yearly county level population data from 1969 to 2015


In [5]:
def readFPISCodes():
    df = pd.read_excel("../Data/US_FIPS_Codes.xls", dtype=object)
    df["COUNTYCODE"] = df["FIPS State"]+df["FIPS County"]
    df["COUNTY"] = df["State"]+","+df["County Name"]
    tmp = df[['COUNTYCODE', 'COUNTY']].copy()
    return tmp

dfFPIS = readFPISCodes()
myDict_FPIS = dfFPIS.set_index('COUNTYCODE').to_dict()
myCountyCodeList = list(dfFPIS["COUNTYCODE"])

In [6]:
# plot the total deaths per year and plot
def getFatalList(myDfList_AccidentData):
    FatalList = []
    YearList = []
    for i in range(len(myDfList_AccidentData)):
        FatalList.append( myDfList_AccidentData[i]["FATALS"].sum()  )
        YearList.append( i+1975 )
    return YearList,FatalList

yearList, fatalList = getFatalList(myDfList_AccidentData)
fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis
# ax.plot(yearList, fatalList)
ax.bar(yearList,fatalList, alpha=0.8,color='#4091e2')
plt.xlabel('Year')
plt.ylabel('Total Deaths/Year')
fig.savefig('../Fig/TotalDeathvsYear.pdf')
plt.close(fig) # close the figure

In [7]:
def mergeAccidentandCrashData(myDfList_AccidentData,myDfList_PopData):
    dfList = []
    for i in range(len(myDfList_AccidentData)):
        tmp = pd.merge(myDfList_AccidentData[i], myDfList_PopData[i], on='COUNTYCODE', how='inner')
        dfList.append(tmp)
    
    return dfList

myDfList_MergedData = mergeAccidentandCrashData(myDfList_AccidentData,myDfList_PopData)
# myDfList_MergedData[0]
#myDfList_PopData[40]
# myDfList_AccidentData[40]

In [41]:
def plotDeathRatePerCounty(myDfList_MergedData,myDict_FPIS,countyCode):
    deathList = []
    yearList = np.arange(1975,2016)
    for i in range(len(myDfList_MergedData)):
        tmp = myDfList_MergedData[i]
        tmp = tmp[ tmp["COUNTYCODE"] == countyCode ]
#         num = 10000*tmp["FATALS"]/tmp["POPULATION"]
        deathList.append( tmp["FATALS"].sum())

    slope, intercept, r_value, p_value, std_err = stats.linregress(yearList,deathList)
    textstr = 'y=%.2f*x + %.2f'%(slope, intercept)
    #print(slope, intercept, r_value, p_value, std_err)
    line = slope*yearList+intercept
    fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis
    ax.plot(yearList,deathList,"ro", yearList, line)
    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    # place a text box in upper left in axes coords
    ax.text(0.55, 0.95, textstr, transform=ax.transAxes, fontsize=14,verticalalignment='top', bbox=props)
    plt.xlabel('Year')
    plt.ylabel('Total Deaths')
    plt.title(myDict_FPIS["COUNTY"][countyCode])
    figName = '../Fig/LinearFitTotalDeaths_'+countyCode+'.pdf'
    fig.savefig(figName)
    plt.close(fig) # close the figure
    
    return slope, intercept



slope, intercept = plotDeathRatePerCounty(myDfList_MergedData,myDict_FPIS,"05005")

crashRatePrediction_2016 = {}
for countyCode in myCountyCodeList:
    slope, intercept = plotDeathRatePerCounty(myDfList_MergedData,myDict_FPIS,countyCode)
    rate = slope*2016+intercept
    crashRatePrediction_2016[countyCode] = rate

In [65]:
def adjustMissingInformation(populationList):
    for i in range(len(populationList)):
        if i==0 and populationList[i]==0:
            populationList[i] = populationList[i+1]
        if i==(len(populationList)-1) and populationList[len(populationList)-1]==0:
            populationList[len(populationList)-1] = populationList[len(populationList)-2]
        if i<(len(populationList)-1) and i > 0 and populationList[i] == 0:
            populationList[i] = (populationList[i-1]+populationList[i+1])/2
    return populationList
    
    


def plotPopIncreasePerCounty(myDfList_MergedData,myDict_FPIS,countyCode):
    yearList = np.arange(1975,2016)
    populationList = np.empty([len(yearList)])
    
    for i in range(len(myDfList_MergedData)):
        tmp = myDfList_MergedData[i]
        tmp = tmp[ tmp["COUNTYCODE"] == countyCode ]
        populationList[i] = tmp["POPULATION"].sum()
    
    # just average the missing information
    populationList = adjustMissingInformation(populationList)
    
    p = np.polyfit(yearList,populationList,5)
    f = np.poly1d(p)

    # calculate new x's and y's
    x_new = np.linspace(yearList[0], yearList[-1], 100)
    y_new = f(x_new)
    

    fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis
    ax.plot(yearList,populationList,"ro", x_new, y_new)
    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    # place a text box in upper left in axes coords
    #ax.text(0.40, 0.15, textstr, transform=ax.transAxes, fontsize=14,verticalalignment='top', bbox=props)
    plt.xlabel('Year')
    plt.ylabel('Total Population')
    plt.title(myDict_FPIS["COUNTY"][countyCode])
    #plt.show()
    figName = '../Fig/LinearPopulationIncreaseFit_'+countyCode+'.pdf'
    fig.savefig(figName)
    plt.close(fig) # close the figure
    
    return f(2016)


num = plotPopIncreasePerCounty(myDfList_MergedData,myDict_FPIS,"37177")
num
# tmp =myDfList_MergedData[40][myDfList_MergedData[40]["COUNTYCODE"]=='37177']
myDfList_MergedData[40]

# populationPrediction_2016 = {}
# for countyCode in myCountyCodeList:
#     num = plotPopIncreasePerCounty(myDfList_MergedData,myDict_FPIS,countyCode)
#     populationPrediction_2016[countyCode] = num

,COUNTYCODE,FATALS,YEAR,POPULATION
0,01001,8,2015,55347
1,01003,32,2015,203709
2,01005,7,2015,26489
3,01007,5,2015,22583
4,01009,5,2015,57673
5,01011,6,2015,10696
6,01013,7,2015,20154
7,01015,15,2015,115620
8,01017,9,2015,34123
9,01019,9,2015,25859


In [52]:
tmp = myDfList_MergedData[11][myDfList_MergedData[11]["COUNTYCODE"] == "05005"]
tmp1= myDfList_PopData[10][myDfList_PopData[10]["COUNTYCODE"] == "05005"]
# myDfList_PopData[10]
#myDfList_MergedData[10]



yearList = np.arange(1975,2016)
populationList = np.empty([len(yearList)])
populationList

# crashRatePrediction_2016
# populationPrediction_2016

df_Crash2016 = pd.DataFrame()
df_Crash2016["CountyCode"] = crashRatePrediction_2016.keys()
df_Crash2016["Fatals"] = crashRatePrediction_2016.values()

df_Population2016 = pd.DataFrame()
df_Population2016["CountyCode"] = populationPrediction_2016.keys()
df_Population2016["Population"] = populationPrediction_2016.values()

In [60]:
df_FinalData =  pd.merge(df_Crash2016, df_Population2016, on='CountyCode', how='inner')
df_FinalData["FatalityRate"] = 10000*df_FinalData["Fatals"]/df_FinalData["Population"]
df_FinalData.to_csv("../CreateSVG/FatalityData.csv",header=False, index = False)